In [30]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [31]:
df = pd.read_csv("data/train.csv")

In [32]:
df.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1181295 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1155051 non-null  float64
 4   Marital Status        1181471 non-null  object 
 5   Number of Dependents  1090328 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            841925 non-null   object 
 8   Health Score          1125924 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       835971 non-null   float64
 12  Vehicle Age           1199994 non-null  float64
 13  Credit Score          1062118 non-null  float64
 14  Insurance Duration    1199999 non-

In [34]:
nulls = df.isnull()

In [35]:
nulls.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [36]:
null_counts = df.isnull().sum()
print(null_counts)

id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64


In [37]:
# Select numeric columns for mean imputation
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Fill NaN values in numeric columns with the mean
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Select categorical columns for encoding
object_cols = df.select_dtypes(include=['object']).columns

# Encode categorical columns using Label Encoding
label_encoders = {}
for col in object_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Convert to string to avoid errors
    label_encoders[col] = le  # Save the encoder for future use

# Alternatively, if you want to fill NaNs in categorical columns with the mode
for col in object_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)  # Fill with mode

# Check the DataFrame after processing
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1200000 non-null  float64
 2   Gender                1200000 non-null  int32  
 3   Annual Income         1200000 non-null  float64
 4   Marital Status        1200000 non-null  int32  
 5   Number of Dependents  1200000 non-null  float64
 6   Education Level       1200000 non-null  int32  
 7   Occupation            1200000 non-null  int32  
 8   Health Score          1200000 non-null  float64
 9   Location              1200000 non-null  int32  
 10  Policy Type           1200000 non-null  int32  
 11  Previous Claims       1200000 non-null  float64
 12  Vehicle Age           1200000 non-null  float64
 13  Credit Score          1200000 non-null  float64
 14  Insurance Duration    1200000 non-

In [38]:
# Normalization for Annual Income and Credit Score
# 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
# Define features and target variable
X = df.drop('Premium Amount', axis=1)  # Features
y = df['Premium Amount']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose a regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmsle = np.sqrt(mean_squared_error(np.log1p(y_test), np.log1p(y_pred)))

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')
print(f'RMSLE: {rmsle}')

Mean Squared Error: 744887.0419032796
R-squared: 0.0032096754783221826
RMSLE: 1.1682038028661481


In [57]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the data
df1 = pd.read_csv("data/test.csv")

# Check the columns in the DataFrame
print("Columns in DataFrame:", df1.columns)

# Select numeric columns for mean imputation
numeric_cols = df1.select_dtypes(include=['float64', 'int64']).columns

# Fill NaN values in numeric columns with the mean
df1[numeric_cols] = df1[numeric_cols].fillna(df1[numeric_cols].mean())

# Select categorical columns for encoding
object_cols = df1.select_dtypes(include=['object']).columns

# Encode categorical columns using Label Encoding
label_encoders = {}
for col in object_cols:
    le = LabelEncoder()
    df1[col] = le.fit_transform(df1[col].astype(str))  # Convert to string to avoid errors
    label_encoders[col] = le  # Save the encoder for future use

# Fill NaNs in categorical columns with the mode
for col in object_cols:
    df1[col].fillna(df1[col].mode()[0], inplace=True)  # Fill with mode

# Check the DataFrame after processing
print(df1.info())

# Check if 'Premium Amount' exists in the DataFrame
if 'Premium Amount' in df1.columns:
    X = df1.drop('Premium Amount', axis=1)  # Features
    y = df1['Premium Amount']  # Target variable
else:
    print("Column 'Premium Amount' not found in DataFrame.")

Columns in DataFrame: Index(['id', 'Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Policy Start Date',
       'Customer Feedback', 'Smoking Status', 'Exercise Frequency',
       'Property Type'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    800000 non-null  int64  
 1   Age                   800000 non-null  float64
 2   Gender                800000 non-null  int32  
 3   Annual Income         800000 non-null  float64
 4   Marital Status        800000 non-null  int32  
 5   Number of Dependents  800000 non-null  float64
 6   Education Level       800000 non-null  int32  
 7   

In [72]:
# Make predictions
y_pred = model.predict(X)
sub1 = pd.DataFrame({id: X.index, "Premium Amount": y_pred})
sub1

,<built-in function id>,Premium Amount
0,0,1200.795667
1,1,1033.476441
2,2,1116.995553
3,3,1068.324417
4,4,1074.128508
...,...,...
1199995,1199995,1138.650052
1199996,1199996,1086.144561
1199997,1199997,1029.006012
1199998,1199998,1142.278562


In [73]:
sub1.to_csv("first_submision_5.csv", index=False)

In [49]:
sub = df[["id", "Premium Amount"]]

In [50]:
sub.to_csv('first_submission.csv', index=False)

In [51]:
len(sub)

1200000

In [52]:
df1 = pd.read_csv("data/test.csv")

In [54]:
len(df1)

800000